<a href="https://colab.research.google.com/github/MZiaAfzal71/Average_Weighted_Path_Vector/blob/main/Data%20Files/Descriptor%20Generators/GenerateDescriptorsData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/MZiaAfzal71/Average_Weighted_Path_Vector.git
!pip install rdkit
%cd Average_Weighted_Path_Vector/Data\ Files

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import rdmolops
from rdkit.Chem import rdFingerprintGenerator
from sklearn.decomposition import PCA

from rdkit.Chem import Descriptors as D
from rdkit.Chem.EState import EState_VSA
from rdkit.Chem import rdMolDescriptors as rdmd

from tqdm.auto import tqdm

import os

# --- PWAV v6: skewness/kurtosis, electronegativity option, top-k, global features ---

# Simple Pauling electronegativity table for common elements (Z -> EN)
_PAULING_EN = {
    1: 2.20,  6: 2.55, 7: 3.04, 8: 3.44, 9: 3.98, 15: 2.19, 16: 2.58, 17: 3.16,
    35: 2.96, 34: 2.66, 14: 1.90, 13: 1.61, 5: 2.04, 3: 0.98, 4: 1.57, 2: 0.0, 33: 2.0,
    11: 0.93, 12: 1.31, 19: 0.82, 20: 1.0, 30: 1.65, 29: 1.9, 8:3.44, 16:2.58, 53: 2.5
}


def mol_from_smile(sm : str) -> Chem.Mol:
      try:
        mol = Chem.MolFromSmiles(sm)  # Convert SMILES to RDKit Mol
        return mol
      except:
        return None


def _en_of_z(z):
    return _PAULING_EN.get(int(z), 2.0)  # default fallback


def _skew_kurt(vals):
    # compute skewness and (excess) kurtosis robustly
    if len(vals) == 0:
        return 0.0, 0.0
    arr = np.asarray(vals, dtype=np.float64)
    m = arr.mean()
    sd = arr.std()
    if sd <= 0:
        return 0.0, 0.0
    skew = ((arr - m) ** 3).mean() / (sd ** 3)
    kurt = ((arr - m) ** 4).mean() / (sd ** 4) - 3.0
    return float(skew), float(kurt)


def build_pwav(bond_type_mat, dist, atomic_numbers, degrees, ssr, aromatic_atoms,
                  L=6, decay=1.0, use_electronegativity=False, topk=0):
    """
    PWAV : enhanced compact descriptor

    Parameters
    ----------
    bond_type_mat, dist, atomic_numbers, degrees : arrays from RDKit
    ssr : iterable of rings from RDKit (Chem.GetSymmSSSR(mol))
    aromatic_atoms : set of aromatic atom indices
    L : int max path length
    decay : float in (0,1] to downweight longer paths by decay**k
    use_electronegativity : bool, if True use Pauling EN differences instead of |Z diff|
    topk : int, include top-k largest path values per bucket (append), 0 disables

    Returns
    -------
    np.ndarray feature vector
    """
    N = len(atomic_numbers)

    # Build ring atom sets and ring-size information
    ring_lists = [list(r) for r in ssr]
    ring_atoms_by_size = {5: [], 6: [], 7: []}
    for ring in ring_lists:
        size = len(ring)
        if size == 5:
            ring_atoms_by_size[5].append(set(ring))
        elif size == 6:
            ring_atoms_by_size[6].append(set(ring))
        elif size >= 7:
            ring_atoms_by_size[7].append(set(ring))

    # per-atom primary ring size bin (0 if none)
    atom_bin = np.zeros(N, dtype=int)
    for sz in (5,6,7):
        for rs in ring_atoms_by_size[sz]:
            for a in rs:
                if atom_bin[a] == 0 or sz < atom_bin[a]:
                    atom_bin[a] = sz

    def categorize_pair(i, j):
        # same ring?
        for sz in (5,6,7):
            for rs in ring_atoms_by_size[sz]:
                if i in rs and j in rs:
                    return f"ring{sz if sz<7 else '7p'}"
        # cross if one in ring
        bi, bj = atom_bin[i], atom_bin[j]
        if (bi in (5,6,7)) ^ (bj in (5,6,7)):
            sz = bi if bi in (5,6,7) else bj
            return f"cross{sz if sz<7 else '7p'}"
        if bi in (5,6,7) and bj in (5,6,7):
            return "cross7p"
        return "nonring"

    buckets_order = ["ring5","ring6","ring7p","cross5","cross6","cross7p","nonring"]

    feats = []

    for k in range(1, L+1):
        buckets = {b: [] for b in buckets_order}
        for i in range(N):
            for j in range(N):
                if i == j: continue
                if dist[i, j] != k: continue
                bond_w = bond_type_mat[i, j] if bond_type_mat[i, j] > 0 else 1
                if use_electronegativity:
                    atom_w = abs(_en_of_z(atomic_numbers[i]) - _en_of_z(atomic_numbers[j]))
                    alpha = 2
                else:
                    atom_w = abs(atomic_numbers[i] - atomic_numbers[j])
                    alpha = 0.5
                deg_w = abs(degrees[i] - degrees[j])
                val = bond_w + alpha * atom_w + 0.3 * deg_w
                if i in aromatic_atoms and j in aromatic_atoms:
                    val *= 1.2
                if decay < 1.0:
                    val *= (decay ** k)
                cat = categorize_pair(i,j)
                buckets[cat].append(val)

        # summarize
        for cat in buckets_order:
            vals = buckets[cat]
            if len(vals) == 0:
                # mean,std,max,skew,kurt -> 5 values
                feats.extend([0.0,0.0,0.0,0.0,0.0])
                if topk > 0:
                    feats.extend([0.0]*topk)
                continue
            arr = np.asarray(vals, dtype=np.float64)
            mean = float(arr.mean())
            sd = float(arr.std())
            mx = float(arr.max())
            skew, kurt = _skew_kurt(arr)
            feats.extend([mean, sd, mx, skew, kurt])
            if topk > 0:
                top_vals = -np.sort(-arr)[:topk]  # descending
                if len(top_vals) < topk:
                    top_vals = np.pad(top_vals, (0, topk - len(top_vals)), constant_values=0.0)
                feats.extend([float(x) for x in top_vals])

    # global descriptors: mean/std of Z and degree
    feats.append(float(np.mean(atomic_numbers)))
    feats.append(float(np.std(atomic_numbers)))
    feats.append(float(np.mean(degrees)))
    feats.append(float(np.std(degrees)))

    # additional global chemical descriptors via RDKit are computed in smiles_to_pwav wrapper
    # pdb.set_trace()
    return np.array(feats, dtype=np.float32)


def smiles_to_pwav(smiles, L=6, decay=1.0, use_electronegativity=False, topk=0):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    # keep hydrogens optional; use heavy-atom graph for many global features
    # but include explicit Hs in adjacency/distance if desired
    mol_h = Chem.AddHs(mol)

    N = mol_h.GetNumAtoms()
    dist = rdmolops.GetDistanceMatrix(mol_h)

    bond_type_mat = np.zeros((N,N), dtype=int)
    for bond in mol_h.GetBonds():
        i,j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        btype = bond.GetBondType()
        if btype.name == 'SINGLE': t=1
        elif btype.name == 'DOUBLE': t=2
        elif btype.name == 'TRIPLE': t=3
        elif btype.name == 'AROMATIC': t=4
        else: t=1
        bond_type_mat[i,j] = t
        bond_type_mat[j,i] = t

    atomic_numbers = np.array([a.GetAtomicNum() for a in mol_h.GetAtoms()])
    degrees = np.array([a.GetDegree() for a in mol_h.GetAtoms()])

    ssr = Chem.GetSymmSSSR(mol_h)
    aromatic_atoms = set([a.GetIdx() for a in mol_h.GetAtoms() if a.GetIsAromatic()])

    base_feats = build_pwav(bond_type_mat, dist, atomic_numbers, degrees, ssr, aromatic_atoms,
                              L=L, decay=decay, use_electronegativity=use_electronegativity, topk=topk)

    return base_feats

# --- Helper: compute extra features depending on property ---
def _count_atom(mol, Z):
    return sum(1 for a in mol.GetAtoms() if a.GetAtomicNum()==Z)


def _add_vsa_features(mol, feats):
    # PEOE_VSA bins (14 bins)
    for i in range(1, 15):
        try:
            val = getattr(D, f"PEOE_VSA{i}")(mol)
        except Exception:
            val = 0.0
        feats[f"PEOE_VSA{i}"] = float(val)

    # Also include the total
    try:
        feats["PEOE_VSA_Total"] = D.PEOE_VSA_Total(mol)
    except Exception:
        feats["PEOE_VSA_Total"] = 0.0

    # EState_VSA bins (11 bins)
    for i in range(1, 12):
        try:
            val = getattr(D, f"EState_VSA{i}")(mol)
        except Exception:
            val = 0.0
        feats[f"EState_VSA{i}"] = float(val)

    # And totals
    feats["EState_VSA_Sum"] = sum(feats[f"EState_VSA{i}"] for i in range(1, 12))
    feats["EState_VSA_Max"] = max(feats[f"EState_VSA{i}"] for i in range(1, 12))


def _largest_fused_aromatic_system_size(mol):
    ri = mol.GetRingInfo()
    a_rings = [set(r) for r in ri.AtomRings() if all(mol.GetAtomWithIdx(a).GetIsAromatic() for a in r)]
    # cluster aromatic rings by shared atoms (simple union-find)
    parents = list(range(len(a_rings)))
    def find(x):
        while parents[x] != x:
            parents[x] = parents[parents[x]]
            x = parents[x]
        return x
    def union(x,y):
        rx, ry = find(x), find(y)
        if rx != ry: parents[ry] = rx

    for i in range(len(a_rings)):
        for j in range(i+1, len(a_rings)):
            if a_rings[i] & a_rings[j]:
                union(i,j)
    clusters = {}
    for i, ring in enumerate(a_rings):
        r = find(i)
        clusters.setdefault(r, set()).update(ring)
    return max((len(v) for v in clusters.values()), default=0), len(clusters)

def _smallest_ring_size(mol):
    # 0 if acyclic
    sizes = [len(r) for r in mol.GetRingInfo().AtomRings()]
    return min(sizes) if sizes else 0

def _charge_summaries(mol):
    # Gasteiger charges: needs hydrogens for better stability
    try:
        mh = Chem.AddHs(mol)
        Chem.rdPartialCharges.ComputeGasteigerCharges(mh)
        qs = [float(a.GetProp('_GasteigerCharge')) for a in mh.GetAtoms()
              if a.HasProp('_GasteigerCharge')]
        if not qs:
            return dict()
        return {
            "Charge_min": float(np.min(qs)),
            "Charge_max": float(np.max(qs)),
            "Charge_std": float(np.std(qs)),
            "AbsCharge_sum": float(np.sum(np.abs(qs))),
            "AbsCharge_mean": float(np.mean(np.abs(qs))),
        }
    except Exception:
        return dict()

def compute_property_specific_features(mol, sheet_name):
    feats = {}
    # additional global RDKit descriptors
    try:
        molwt = D.MolWt(mol)
    except Exception:
        molwt = float(Chem.Descriptors.MolWt(mol)) if hasattr(Chem,'Descriptors') else 0.0
    try:
        tpsa = rdmd.CalcTPSA(mol)
    except Exception:
        tpsa = 0.


    num_atoms = mol.GetNumAtoms()
    num_arom = sum(1 for a in mol.GetAtoms() if a.GetIsAromatic())
    frac_arom = float(num_arom) / float(num_atoms) if num_atoms > 0 else 0.0
    formal_charge = Chem.GetFormalCharge(mol) if hasattr(Chem, 'GetFormalCharge') else 0

    feats["MolWt"] = molwt
    feats["TPSA"] = tpsa
    feats["nAtoms"] = num_atoms
    feats["fCharge"] = formal_charge

    # return np.concatenate([base_feats, extra])
    HAC = mol.GetNumHeavyAtoms() or 1

    # --- universal additions (independent of sheet_name) ---
    # Crippen polarizability (MolMR) and Bertz complexity
    feats["MolMR"] = D.MolMR(mol)
    feats["BertzCT"] = D.BertzCT(mol)

    # Surface-related descriptors
    feats["LabuteASA"] = rdmd.CalcLabuteASA(mol)

    # EState/PEOE VSA partitions (summed by RDKit)
    # These return a list across bins; keep a compact summary: totals and extremes
    # peoe_bins = rdmd.CalcPEOE_VSA_(mol)  # internal; robust alternative:
    try:
        _add_vsa_features(mol, feats)     # single float (total)
        # feats["PEOE_VSA_Total"] = float(peoe)
    except Exception:
        pass
    try:
        est_bins = EState_VSA.EState_VSA_(mol)  # list across bins
        feats["EState_VSA_Sum"] = float(np.sum(est_bins))
        feats["EState_VSA_Max"] = float(np.max(est_bins)) if len(est_bins) else 0.0
    except Exception:
        pass

    # Ring system descriptors
    ri = mol.GetRingInfo()
    n_rings = ri.NumRings()
    feats["RingCount"] = n_rings
    feats["AromaticRings"] = sum(
        1 for r in ri.AtomRings() if all(mol.GetAtomWithIdx(a).GetIsAromatic() for a in r)
    )
    feats["HeteroRings"] = sum(
        1 for r in ri.AtomRings() if any(mol.GetAtomWithIdx(a).GetAtomicNum() not in (6,1) for a in r)
    )
    feats["SmallestRing"] = _smallest_ring_size(mol)
    largest_aro_sys, n_aro_sys = _largest_fused_aromatic_system_size(mol)
    feats["LargestAroSystemSize"] = largest_aro_sys
    feats["NumAroSystems"] = n_aro_sys

    # Flexibility / density
    feats["RotBondsPerHA"] = rdmd.CalcNumRotatableBonds(mol) / HAC
    feats["HBondSitesPerHA"] = (rdmd.CalcNumHBD(mol) + rdmd.CalcNumHBA(mol)) / HAC

    # Functional fragments (a few high-impact ones)
    patt_smarts = {
        "nitrile": "C#N",
        "amide": "C(=O)N",
        "acid": "C(=O)[OH]",
        "ester": "C(=O)O[C;!H0]",  # crude, avoids carboxylate
        "aniline": "Nc1ccccc1",
        "nitro": "[$([NX3](=O)=O),$([NX3+](=O)[O-])]",
        "sulfonamide": "S(=O)(=O)N",
        "quat_ammonium": "[N+](C)(C)(C)C",
    }
    for name, sm in patt_smarts.items():
        patt = Chem.MolFromSmarts(sm)
        feats[f"frag_{name}"] = len(mol.GetSubstructMatches(patt)) if patt is not None else 0

    # Partial charges (Gasteiger)
    feats.update(_charge_summaries(mol))

    # --- keep your sheet-specific blocks too (they stay additive) ---
    if sheet_name == "LogBCF":
        logp = D.MolLogP(mol)
        tpsa = rdmd.CalcTPSA(mol)
        hbd  = rdmd.CalcNumHBD(mol)
        hba  = rdmd.CalcNumHBA(mol)
        frac_csp3 = rdmd.CalcFractionCSP3(mol)
        mw = D.MolWt(mol)
        k2 = D.Kappa2(mol)
        bj = D.BalabanJ(mol)

        feats.update({
            "MolLogP": logp,
            "MolLogP2": logp*logp,
            "TPSA": tpsa,
            "TPSA_per_HA": tpsa / HAC,
            "HBD": hbd,
            "HBA": hba,
            "HBDxHBA": hbd * hba,
            "FractionCSP3": frac_csp3,
            "MolWt": mw,
            "Kappa2": k2,
            "BalabanJ": bj,
            "nF": _count_atom(mol, 9),
            "nCl": _count_atom(mol, 17),
            "nBr": _count_atom(mol, 35),
            "nI": _count_atom(mol, 53),
            "nHetero": sum(1 for a in mol.GetAtoms() if a.GetAtomicNum() not in (1,6))
        })

    elif sheet_name == "MP":
        hbd  = rdmd.CalcNumHBD(mol)
        hba  = rdmd.CalcNumHBA(mol)
        rot  = rdmd.CalcNumRotatableBonds(mol)
        frac_csp3 = rdmd.CalcFractionCSP3(mol)
        ring_info = mol.GetRingInfo()
        n_rings = ring_info.NumRings()
        n_arom_rings = sum(
            1 for r in ring_info.AtomRings()
            if all(mol.GetAtomWithIdx(a).GetIsAromatic() for a in r)
        )
        k2 = D.Kappa2(mol)
        k3 = D.Kappa3(mol)
        bj = D.BalabanJ(mol)

        morgan_gen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)
        fp = morgan_gen.GetFingerprint(mol)
        envs = set(fp.GetOnBits())
        sym_proxy = len(envs) / HAC  # lower → more symmetry

        feats.update({
            "HBD": hbd,
            "HBA": hba,
            "HBDplusHBA": hbd + hba,
            "HBDxHBA": hbd * hba,
            "RotBonds": rot,
            "FractionCSP3": frac_csp3,
            "RingCount": n_rings,
            "AromaticRingCount": n_arom_rings,
            "AromaticRingFrac": (n_arom_rings / n_rings) if n_rings else 0.0,
            "Kappa2": k2,
            "Kappa3": k3,
            "BalabanJ": bj,
            "SymmetryProxy": sym_proxy
        })

    return feats



def build_dataset(data_df, sheet, outfile="pwav_v8_dataset.csv", L=6, decay=1.0,
                  use_electronegativity=False, topk=0, compress_dim=None):
    rows = []
    indices = []
    extra_features = []

    smiles_list = data_df["SMILES"]
    for i, s in tqdm(enumerate(smiles_list), total=len(smiles_list)):
        v = smiles_to_pwav(s, L=L, decay=decay, use_electronegativity=use_electronegativity, topk=topk)
        if v is None:
            continue
        rows.append(v)
        indices.append(i)

        mol = Chem.MolFromSmiles(s)
        if mol is not None:
            extra_features.append(compute_property_specific_features(mol, sheet))
        else:
            extra_features.append({})

    X = np.vstack(rows)

    # pdb.set_trace()
    # Optional PCA compression
    if compress_dim is not None and compress_dim < X.shape[1]:
        pca = PCA(n_components=compress_dim)
        Xr = pca.fit_transform(X)
        df = pd.DataFrame(Xr)
    else:
        df = pd.DataFrame(X)

    # Rename columns with sheet prefix
    new_col_dict = {col: f"{sheet}_{col}" for col in df.columns}
    df.rename(columns=new_col_dict, inplace=True)

    # Append extra RDKit features (if any)
    ef_df = pd.DataFrame(extra_features).fillna(0)
    ef_col_dict = {col: f"{sheet}_{col}" for col in ef_df.columns}
    ef_df.rename(columns=ef_col_dict, inplace=True)

    new_df = pd.concat([data_df.loc[indices], df, ef_df], axis=1)
    # new_df = pd.concat([data_df.loc[indices], df], axis=1)
    new_df.to_parquet(outfile, index=False)

    print(f"Saved {outfile} shape={new_df.shape} (decay={decay}, use_en={use_electronegativity}, topk={topk})")
    return new_df


In [ ]:
input_file = "Excel Files/Zang_Data.xlsx"
property = ["Log VP", "MP", "BP", "LogBCF", "LogS", "LogP"]

output_dir = "Descriptors Data"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
for prop in property:
    df = pd.read_excel(input_file, sheet_name=prop)
    df['Preferred_name'] = df['Preferred_name'].astype(str)

    print(f"\nProcessing SMILES from sheet:{prop} .... \n")


    output_file = os.path.join(output_dir, f"{prop}_pwav.parquet")
    build_dataset(df, prop, output_file, L=5, decay=0.5,
                       use_electronegativity=True, topk=5, compress_dim=64)



print(f'\nAll sheets have been processed successfully!')